# Project: MoonShot - AI-powered Trading Strategy

This notebook outlines the development of MoonShot, an Artificial intelligence (AI) system designed to assess the increase in a single stock price against a wide array of technical indicators and use this information to develop a trading pattern. We aim to achieve the following objectives:

---

### Select effective technical indicators:
We will feed a number of technical indicators into the AI as additional training parameters, it will determine correlations between indicators and the increase in stock price.
From these technical indicator correlations we will select the most effective techincal indicators with the most correlation to a rise in stock price. 

---

### Train a Neural Network:
We will train a multilayer neural network on historical market data and the corresponding stock price history as well as the technical indicators selected. This neural network will attempt to learn and potentially improve upon the initial strategy, potentially identifying new patterns or refining existing ones.

We will consider the model a success if it is able to increase both the Win rate by 15% while maintaining the profit percentage.

#### Throughout this notebook, we will document the development process, including:

- Data acquisition and preparation
- Feature engineering and selection
- Building and training the neural network
- Evaluating the performance of MoonShot's strategy and the trained neural network

---

Disclaimer: This project is for educational purposes only and should not be used for real-world trading without proper risk management and regulatory compliance. The market is inherently risky, and any trading strategy, including those involving AI, is susceptible to losses.

# Import Required Libraries

In [3]:
# Required libraries

# Import the generic libraries
import sys
import pytictoc

#Import the neural network architecture
import torch
import torch.nn as nn
import torch.optim as optim

#Import financial data
import ta
import yfinance as yf

# Import data science tools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tickers500 import Tickers500
from tickerTA import Ticker
from tickerTA import TechnicalAnalysis

In [4]:
print(torch.__version__)

2.2.2+cpu


In [ ]:
print(torch.cuda.is_available())

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.is_available())

In [ ]:

cuda = torch.device('cuda')
cuda
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device

In [ ]:
torch.cuda.is_available()

In [ ]:
# Install torch with cuda support
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

---
# Load and Preprocess Data

## 1. Initialize our Tickers500 class, update all TickerData stored CSV files, import the DataFrame we are interested in from CSV

In [ ]:
tickers500 = Tickers500()
ticker = tickers500.get_random_ticker()
ticker


In [ ]:
start_date = '2020-01-01'
ticker_df = tickers500.load_ticker_data_to_df(ticker, start_date)
ticker_df

## 2. Send that DataFrame through TechnicalAnalysis class to have the technical indicators added on to a DataFrame.

In [ ]:
technical_analysis = TechnicalAnalysis(ticker, ticker_df)
technical_analysis.df_ta

In [ ]:
technical_analysis.plot_bollinger_bands()

In [ ]:
technical_analysis.df_ta.drop(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

### preprocess the main dataset

* Drop data not required by any subsets
* Split the data into the X and Y sets
* Handle missing data
* scaling, normalization, and correlations




```
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        #define the first layer which has neurons = <input_size> with edges per neuron = <hidden_size>
        self.fc0 = nn.Linear(input_size, hidden_size)
        #defines the hidden layers with <hidden_size> neurons and <hidden_size> outgoing edges
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        # defines the second hidden layer
        #self.fc2 = nn.Linear(hidden_size, hidden_size)
        # defines the third hidden layer
        #self.fc3 = nn.Linear(hidden_size, hidden_size)
        #defines the output layer with hidden_size connections going to output_size neurons
        self.fcf = nn.Linear(hidden_size, output_size)
        #defines the relu function used as the activation function between neurons
        self.relu = nn.ReLU()
        #defines the final function used on the forward pass
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()


    def forward(self, x):
        x = self.fc0(x)
        x = self.relu(x)

        x = self.fc1(x)
        x = self.relu(x)

        #x = self.fc2(x) # Second hidden layer
       # x = self.relu(x)

       # x = self.fc3(x) # Third hidden layer
       # x = self.relu(x)

        x = self.fcf(x)
        x = self.sigmoid(x)
        #x = self.softmax(x)
        #x = x.squeeze(1) # added to remove additional dimension [400, 1] added during pytorch linear layering removed for softmax
        return x

## Added a second layer with lines
#         self.fc2a = nn.Linear(hidden_size, hidden_size)
#        x = self.relu(x)
#        x = self.fc2a(x)
```

In [ ]:
# Two new dataframes are created, one contains all of the input features of the dataset (x), the other contains the target values (y)
x = technical_analysis.df_ta.drop(columns=['Adj Close'])
y = technical_analysis.df_ta['Adj Close']

# The data is split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# The data is scaled
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# The data is converted to PyTorch tensors
x_train = torch.FloatTensor(x_train).to('cuda')
x_test = torch.FloatTensor(x_test).to('cuda')
y_train = torch.FloatTensor(y_train.to_numpy()).to('cuda')
y_test = torch.FloatTensor(y_test.to_numpy()).to('cuda')

# The neural network is defined
class ANN(nn.Module):
    def __init__(self, input_features=9, hidden1=20, hidden2=20, output_features=1):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, output_features)
    def forward(self, x):
        x = torch.relu(self.f_connected1(x))
        x = torch.relu(self.f_connected2(x))
        x = self.out(x)
        return x
    
# The model is instantiated
torch.manual_seed(20)
model = ANN()
model.to('cuda')

# The loss function and optimizer are defined
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [ ]:

# The model is trained
epochs = 500
final_losses = []
t = pytictoc.TicToc()
t.tic()

for i in range(epochs):
    i += 1
    y_pred = model.forward(x_train)
    loss = loss_function(y_pred, y_train)
    final_losses.append(loss)
    if i%10 == 1:
        print('Epoch number: {} and the loss: {}'.format(i, loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

t.toc()

## Training Setup

> We attempt to convert the numpy and pandas series we have currently used for our dataset into tensors
Pandas dataframes and Numpy Arrays are used before this step for data exploration and manipulation but the deep learning library pytorch performs operations on tensors.

## Training and Tuning

> Here we define the hyperparameters of the neural network and begin training the network with those parameters. As deep learning is an iterative process- with model degredation and improvements both contributing to overall progress- this section does not contain the history of experimental training and parameter tuning that moonShot has undergone.